## 2017, 2018 미세먼지 데이터 전처리
미세먼지 데이터는 두 개의 폴더로 나누어져 있습니다.  
각각에 폴더에는 분기별로 csv 파일이 저장되어 있습니다. (한 폴더당 4개씩 총 8개)  
아래의 함수는, 각각의 폴더에 접근하여 미세먼지 데이터들을 전처리 하는 코드입니다.   
전처리가 끝난 코드는 """미세먼지 전처리""" 라는 이름을 가진 폴더로 이동하게 됩니다.  
코드를 수정해야한다면, 데이터들이 들어있는 data_dir를 수정하면 작동합니다.  



In [1]:
# import moduel
import pandas as pd # pandas 모듈
import numpy as np # numpy 모듈
import os # 폴더를 생성하고 다루기 위함
from os import listdir # 폴더 리스트를 불러오기 위함
import re # 정규표현식 사용

def DataPreprocessing(dir_name): # 폴더이름을 인자로 받는 데이터 전처리 함수
    # 데이터가 들어있는 폴더경로
    # '2017' 데이터 폴더에는 1분기, 2분기, 3분기, 4분기의 미세먼지 데이터가 들어있다.
    data_dir = "C:\\Users\\User\\Documents\\data_mise\\{}".format(dir_name)
    # data_dir로 주피터가 작업하는 폴더를 인식하도록 경로 변경
    working_dir = os.chdir(data_dir) 

    # 전처리된 데이터를 내보낼 폴더
    out_dir = "C:\\Users\\User\\Documents\\data_mise\\{}\\".format('미세먼지_전처리') 
    if not os.path.exists(out_dir): # 만약 경로가 없으면, 
            os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.

    # 전처리할 파일들이 들어있는 폴더의 파일 목록을 list형태로 추출
    file_list = os.listdir(data_dir)
    # 최종적으로 합쳐질 빈 데이터 프레임 생성
    Merged_DF = pd.DataFrame(columns=['측정소코드', '측정소명', '측정일시', 
                                      'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25'])

    # for문을 돌면서 하나씩 파일을 꺼내옴
    for file in file_list:
        raw_df = pd.read_csv(file, encoding='cp949')
        raw_df = raw_df.rename(columns={'Unnamed: 0': '지역'})
        raw_df = raw_df.loc[:, ~raw_df.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제

        # 분석에 사용하지 않을 ['주소'] columns drop
        raw_df = raw_df.drop(['주소'], axis='columns')

        # 데이터 "결측치" 처리 code -> Not a Number로 변경
        raw_df = raw_df.replace([-999,-9999],[np.nan, np.nan])

        # pm10의 이상치 제거
        # 데이터 "이상치" 처리 code -> 바로 앞의 값으로 대체하는 논리구조
        raw_df[raw_df['PM10'] > 1050.0].index
        outlier = raw_df[raw_df['PM10'] > 1050.0].index
        raw_df['PM10'].loc[outlier] = raw_df['PM10'].loc[outlier-1]

        # "서울 xxx" 와 같은 지역을 찾아내기 위해 '서울'이 포함된 지역만 검색
        raw_df = raw_df[raw_df['지역'].str.contains('서울')]

        # datetime형식으로 측정일시 type을 변경
        raw_df['측정일시'] = raw_df['측정일시'].astype('str')
        date_ls = list(raw_df['측정일시']) # 날짜 데이터를 list형태로 변경

        # 시간별로 date를 절사. :13까지가 시간단위!
        days_ls = []
        for date in date_ls:
            date = date[:8]
            days_ls.append(date)
        raw_df['측정일시'] = days_ls

        # 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
        raw_df['측정일시'] = pd.to_datetime(raw_df['측정일시'], format='%Y%m%d')

        # 위에서 생성한 빈 데이터 프레임에 연결
        Merged_DF = pd.concat([Merged_DF, raw_df])
        Merged_DF = Merged_DF.reset_index(drop=True) # 인덱스 초기화

    #     print(len(raw_df)) # 분기별 데이터 수 확인
    #     print(raw_df['PM10'].max()) # 이상치 확인을 위함
    #     print(raw_df['PM25'].min()) # 이상치 확인을 위함 

    # 합쳐진 df을 다른 이름으로 빼내기
    print(Merged_DF.head()) # 데이터 확인 
    Merged_DF = Merged_DF.sort_values(by='지역') # 지역별로 오름차순 정렬
    Merged_DF = Merged_DF.reset_index(drop=True) # index 초기화
    Merged_DF = round(Merged_DF.groupby(['측정일시', '지역']).mean(),2) # 측정일시와 지역으로 groupby
    print(len(Merged_DF)) # 데이터 개수 확인 
    Merged_DF.to_csv(out_dir + 'total_{}.csv'.format(dir_name), encoding='cp949') # 데이터 쓰기

In [2]:
DataPreprocessing('2017')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



    CO    NO2     O3  PM10  PM25    SO2     지역 측정소명   측정소코드       측정일시
0  1.3  0.068  0.002  77.0  63.0  0.006  서울 중구   중구  111121 2017-01-01
1  1.4  0.066  0.002  76.0  63.0  0.006  서울 중구   중구  111121 2017-01-01
2  1.2  0.063  0.002  73.0  57.0  0.005  서울 중구   중구  111121 2017-01-01
3  1.1  0.053  0.002  67.0  55.0  0.005  서울 중구   중구  111121 2017-01-01
4  1.1  0.051  0.002  66.0  54.0  0.004  서울 중구   중구  111121 2017-01-01
9855


In [3]:
DataPreprocessing('2018')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



    CO    NO2     O3  PM10  PM25    SO2    망     지역 측정소명   측정소코드       측정일시
0  0.5  0.020  0.020  34.0  19.0  0.004  NaN  서울 중구   중구  111121 2018-01-01
1  0.4  0.016  0.024  27.0  14.0  0.004  NaN  서울 중구   중구  111121 2018-01-01
2  0.4  0.022  0.018  26.0  14.0  0.004  NaN  서울 중구   중구  111121 2018-01-01
3  0.5  0.030  0.010  26.0  15.0  0.004  NaN  서울 중구   중구  111121 2018-01-01
4  0.6  0.029  0.011  28.0  16.0  0.004  NaN  서울 중구   중구  111121 2018-01-01
9916


## 전처리가 끝난 2017, 2018 미세먼지 데이터 합치기
미세먼지외, 다른 데이터들과의 원활한 결합을 위하여 데이터 값들과 column의 이름을 통일화

In [4]:
data_dir = "C:\\Users\\User\\Documents\\data_mise\\미세먼지_전처리" # data가 들어있는 폴더 경로
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

_2017 = pd.read_csv('total_2017.csv',encoding='cp949') # 데이터 읽기
_2018 = pd.read_csv('total_2018.csv',encoding='cp949') # 데이터 읽기

_2017['지역'] = _2017['지역'].str.replace('서울 ', '') # 앞부분의 '서울 ' 제거. 구 이름만 남김
_2018['지역'] = _2018['지역'].str.replace('서울 ', '') # 앞부분의 '서울 ' 제거. 구 이름만 남김 

_2years = _2017.append(_2018) # 2017에 2018 데이터를 append
_2years = _2years.reset_index(drop=True) # 인덱스 초기화
_2years = _2years.rename(columns={'측정일시':'date'}) # 다른 데이터와의 결합을 위해 이름 변경
_2years.head()

,date,지역,CO,NO2,O3,PM10,PM25,SO2
0,2017-01-01,강남구,0.98,0.04,0.00,78.56,57.79,0.01
1,2017-01-01,강동구,0.88,0.04,0.00,81.25,54.21,0.00
2,2017-01-01,강북구,0.98,0.05,0.00,78.17,65.21,0.00
3,2017-01-01,강서구,0.90,0.06,0.00,84.08,NaN,0.01
4,2017-01-01,강서로,0.96,0.05,0.01,75.12,53.64,0.01


In [5]:
# 금천구의 경우 데이터가 중복
# 중구 청의 경우, 중구 청계천을 의미하는데, 중구라는 구가 있으므로 제외
# 강서로의 경우, 강서로를 의미하는데, 강서구라는 구가 있으므로 제외 
_2years = _2years[_2years['지역'] != '금천구']
_2years = _2years[_2years['지역'] != '중구 청']
_2years = _2years[_2years['지역'] != '강서로 ']

In [6]:
_2years.to_csv('2017X2018_서울미세먼지_합칠용.csv', encoding='cp949')